In [1]:
import pandas as pd
import numpy as np

In [2]:
# Pour un DataFrame comme ceci :
df = pd.DataFrame(
    {
        "filiere": pd.Series(['GCPE','GCPE','GCPE','MAR','VITI','MAR','HORTI','GCPE','GCPE']),
        "espece": pd.Series(['Maïs','Blé','Blé','Carotte','Vigne','Carotte','Tulipe','Moutarde','Luzerne'], dtype="string"),
        "culture": pd.Series(['Cereale','Cereale','Cereale','Légume',np.nan,'Légume','Ornement',np.nan,'Fourrage'], dtype="string")
    }
)

df

,filiere,espece,culture
0,GCPE,Maïs,Cereale
1,GCPE,Blé,Cereale
2,GCPE,Blé,Cereale
3,MAR,Carotte,Légume
4,VITI,Vigne,<NA>
5,MAR,Carotte,Légume
6,HORTI,Tulipe,Ornement
7,GCPE,Moutarde,<NA>
8,GCPE,Luzerne,Fourrage


In [3]:
# On crée d'abord les fonction que l'on veut réutiliser dans le groupby

def concat_unique_sorted_txt(series):
    '''
    Celle_ci permet de retourner une concaténation textuelle des valeurs uniques de la colonne
    Prends en charge les NaN et trie les valeurs à concaténer par ordre alphabetique
    '''
    cleaned = series.dropna().unique()
    if len(cleaned) == 0:
        return np.nan
    return '_'.join(sorted(cleaned))

def get_nb_unique_txt(series):
    '''
    Celle_ci permet de retourner le nombre de valeur unique, non-NaN d'un groupe
    '''
    cleaned = series.dropna().unique()
    return len(cleaned)

In [7]:
# On crée des colonnes que l'on veut ajouter
# On veut le nombre d'occurence de chaque groupe
df['nb_occur_grp'] = 1
# Par exemple pour avoir le nombre de typologie de culture différentes sans pour autant écraser la colonne 'espece', on la duplique
df['nb_espece'] = df['espece']
# Pareil pour culture
df['nb_culture'] = df['culture']
# Pour tester on regarde les stats sur la longeur des noms d'espece
df['test'] = df['espece'].str.len()

In [8]:
# On fait un dictionnaire qui reprends les colonnes qui vont utiliser tel ou tel fonction
agg_dict = {
    'espece': concat_unique_sorted_txt,
    'culture': concat_unique_sorted_txt,
    'nb_occur_grp' : 'sum',
    'nb_espece': get_nb_unique_txt,
    'nb_culture': get_nb_unique_txt,
    'test': ['mean','max','min'] # Multi-index de colonne possible
}

In [9]:
# On groupe par filiere. On n'oublie aucune colonne et on reset l'index à la fin
df_final = df[['filiere','espece','culture','nb_occur_grp','nb_espece','nb_culture','test']].groupby('filiere')\
    .agg(agg_dict).reset_index().set_index('filiere')

df_final

espece                  culture nb_occur_grp  \
          concat_unique_sorted_txt concat_unique_sorted_txt          sum   
filiere                                                                    
GCPE     Blé_Luzerne_Maïs_Moutarde         Cereale_Fourrage            5   
HORTI                       Tulipe                 Ornement            1   
MAR                        Carotte                   Légume            2   
VITI                         Vigne                     <NA>            1   

                nb_espece        nb_culture test          
        get_nb_unique_txt get_nb_unique_txt mean max min  
filiere                                                   
GCPE                    4                 2  5.0   8   3  
HORTI                   1                 1  6.0   6   6  
MAR                     1                 1  7.0   7   7  
VITI                    1                 0  5.0   5   5